In [1]:
from xdsl.dialects.experimental import aie
from xdsl.dialects.builtin import Region, IndexType, ModuleOp, i32, IntegerAttr, ArrayAttr, i64, StringAttr
from xdsl.dialects.arith import Constant
from xdsl.dialects.memref import MemRefType
from xdsl.dialects import builtin, arith, memref, func
from xdsl.builder import Builder
from xdsl.traits import SymbolTable

def I32Attr(value):
    return IntegerAttr.from_int_and_width(value, i32)

In [2]:
def tutorial6():
    tile14 = aie.TileOp(I32Attr(1), I32Attr(4))
    tile34 = aie.TileOp(I32Attr(3), I32Attr(4))

    buf14 = aie.BufferOp(tile14, i32, ArrayAttr([I32Attr(256)]), StringAttr("a14"))
    buf34 = aie.BufferOp(tile34, i32, ArrayAttr([I32Attr(256)]), StringAttr("a34"))

    lock14_6 = aie.LockOp(I32Attr(6), I32Attr(0), tile14, StringAttr("lock_a14_6"))
    lock34_7 = aie.LockOp(I32Attr(7), I32Attr(0), tile34, StringAttr("lock_a34_7"))
    lock34_8 = aie.LockOp(I32Attr(8), I32Attr(0), tile34, StringAttr("lock_a34_8"))

    @Builder.region
    def packet_flow_region(builder: Builder):
        packet_source = aie.PacketSourceOp(aie.WireBundleAttr("DMA"), I32Attr(0), tile14)
        packet_dest = aie.PacketDestOp(aie.WireBundleAttr("DMA"), I32Attr(1), tile34)
        builder.insert(packet_source)
        builder.insert(packet_dest)

    packet_flow = aie.PacketFlowOp(I32Attr(0xD), packet_flow_region)

    # CORE 14
    uselock14_6_acquire = aie.UseLockOp(I32Attr(0), I32Attr(aie.LOCK_ACQUIRE), I32Attr(aie.BLOCKING), lock14_6)
    
    val = arith.Constant.from_int_and_width(14, i32)
    idx = arith.Constant.from_int_and_width(3, IndexType())
    memref_store = memref.Store.get(val, buf14, idx)

    uselock14_6_release = aie.UseLockOp(I32Attr(1), I32Attr(aie.LOCK_RELEASE), I32Attr(aie.BLOCKING), lock14_6)

    @Builder.region
    def core14_region(builder: Builder):
        builder.insert(uselock14_6_acquire)
        builder.insert(val)
        builder.insert(idx)
        builder.insert(memref_store)
        builder.insert(uselock14_6_release)
        builder.insert(aie.EndOp())

    core14 = aie.CoreOp(I32Attr(1), tile14, core14_region)

    # MEM 14
    uselock14_6_acquire_mem = aie.UseLockOp(I32Attr(1), I32Attr(aie.LOCK_ACQUIRE), I32Attr(aie.BLOCKING), lock14_6)
    uselock14_6_release_mem = aie.UseLockOp(I32Attr(1), I32Attr(aie.LOCK_RELEASE), I32Attr(aie.BLOCKING), lock14_6)
    dma_block = builtin.Block()
    dma_packet = aie.DMABDPACKETOp(I32Attr(0x4), I32Attr(0xD))
    dmabd_buf14 = aie.DMABDOp(I32Attr(0), I32Attr(256), I32Attr(0), None, buf14)


    chain = builtin.Block([uselock14_6_acquire_mem, dma_packet, dmabd_buf14, uselock14_6_release_mem])
    dest = builtin.Block([aie.EndOp()])

    chain.add_op(aie.NextBDOp(dest))


    mem14 = aie.MemOp(tile14, Region([dma_block,chain,dest]))
    

    dma_start = aie.DMAStartOp(I32Attr(aie.MM2S), I32Attr(0), chain, dest)

    dma_block.add_op(dma_start)
    

    # CORE 34
    uselock34_8_acquire_core = aie.UseLockOp(I32Attr(0), I32Attr(aie.LOCK_ACQUIRE), I32Attr(aie.BLOCKING), lock34_8)
    uselock34_7_acquire_core = aie.UseLockOp(I32Attr(1), I32Attr(aie.LOCK_ACQUIRE), I32Attr(aie.BLOCKING), lock34_7)
    idx1 = arith.Constant.from_int_and_width(3, IndexType())
    d1 = memref.Load.get(buf34, idx1)
    c1 = arith.Constant.from_int_and_width(100, i32)
    d2 = arith.Addi(d1, c1)
    idx2 = arith.Constant.from_int_and_width(5, IndexType())
    memref_store_d2_in = memref.Store.get(d2, buf34, idx2)
    uselock34_7_release_core = aie.UseLockOp(I32Attr(0), I32Attr(aie.LOCK_RELEASE), I32Attr(aie.BLOCKING), lock34_7)
    uselock34_8_release_core = aie.UseLockOp(I32Attr(1), I32Attr(aie.LOCK_RELEASE), I32Attr(aie.BLOCKING), lock34_8)

    @Builder.region
    def core34_region(builder: Builder):
        builder.insert(uselock34_8_acquire_core)
        builder.insert(uselock34_7_acquire_core)
        builder.insert(idx1)
        builder.insert(d1)
        builder.insert(c1)
        builder.insert(d2)
        builder.insert(idx2)
        builder.insert(memref_store_d2_in)
        builder.insert(uselock34_7_release_core)
        builder.insert(uselock34_8_release_core)
        builder.insert(aie.EndOp())

    core34 = aie.CoreOp(I32Attr(1), tile34, core34_region)

    # MEM 34
    uselock34_7_acquire_mem = aie.UseLockOp(I32Attr(0), I32Attr(aie.LOCK_ACQUIRE), I32Attr(aie.BLOCKING), lock34_7)
    dmabd_buf34 = aie.DMABDOp(I32Attr(0), I32Attr(256), I32Attr(0), None, buf34)
    uselock34_7_release_mem = aie.UseLockOp(I32Attr(1), I32Attr(aie.LOCK_RELEASE), I32Attr(aie.BLOCKING), lock34_7)
    chain = builtin.Block([uselock34_7_acquire_mem, dmabd_buf34, uselock34_7_release_mem])
    dest = builtin.Block([aie.EndOp()])
    chain.add_op(aie.NextBDOp(dest))

    dma_block = builtin.Block()
    mem34 = aie.MemOp(tile34, Region([dma_block,chain,dest]))

    dma_start = aie.DMAStartOp(I32Attr(aie.S2MM), I32Attr(1), chain, dest)

    dma_block.add_op(dma_start)


    @Builder.region
    def region0(builder: Builder):
        builder.insert(tile14)
        builder.insert(tile34)
        builder.insert(buf14)
        builder.insert(buf34)
        builder.insert(lock14_6)
        builder.insert(lock34_7)
        builder.insert(lock34_8)
        builder.insert(packet_flow)
        builder.insert(core14)
        builder.insert(mem14)
        builder.insert(core34)
        builder.insert(mem34)

    device = aie.DeviceOp(I32Attr(0), region0)
    module = ModuleOp([device])

    print(module)

In [3]:
tutorial6()

builtin.module {
  AIE.device(xcvc1902) {
    %0 = "AIE.tile"() {"col" = 1 : i32, "row" = 4 : i32} : () -> index
    %1 = "AIE.tile"() {"col" = 3 : i32, "row" = 4 : i32} : () -> index
    %2 = "AIE.buffer"(%0) {"sym_name" = "a14"} : (index) -> memref<256xi32>
    %3 = "AIE.buffer"(%1) {"sym_name" = "a34"} : (index) -> memref<256xi32>
    %4 = "AIE.lock"(%0) {"lockID" = 6 : i32, "init" = 0 : i32, "sym_name" = "lock_a14_6"} : (index) -> index
    %5 = "AIE.lock"(%1) {"lockID" = 7 : i32, "init" = 0 : i32, "sym_name" = "lock_a34_7"} : (index) -> index
    %6 = "AIE.lock"(%1) {"lockID" = 8 : i32, "init" = 0 : i32, "sym_name" = "lock_a34_8"} : (index) -> index
    AIE.packet_flow(0xD) {
      AIE.packet_source<%0, DMA : 0>
      AIE.packet_dest<%1, DMA : 1>
    }
    %7 = AIE.core (%0) {
      AIE.useLock(%4, "Acquire", 0)
      %8 = arith.constant 14 : i32
      %9 = arith.constant 3 : index
      memref.store %8, %2[%9] : memref<256xi32>
      AIE.useLock(%4, "Release", 1)
      "AIE.end"(